# IpynbCompile.jl: 一个实用的「Jupyter笔记本→Julia源码」转换小工具

<!-- README-ignored -->
（✨执行其中所有单元格，可自动构建、测试并生成相应`.jl`源码！）

## 主要功能

### 简介

📍主要功能：**编译转换**&**解释执行** [***Jupyter***](https://jupyter.org/) 笔记本（`.ipynb`文件）

- 📌可【打开】并【解析】Jupyter笔记本：提供基本的「Jupyter笔记本」「Jupyter笔记本元数据」「Jupyter笔记本单元格」数据结构定义
    - 笔记本 `IpynbNotebook{单元格类型}`
    - 元数据 `IpynbNotebookMetadata`
    - 单元格 `IpynbCell`
- 📌可将Jupyter笔记本（`.ipynb`文件）【转换】成可直接执行的 [***Julia***](https://julialang.org/) 代码
    - 编译单元格 `compile_cell`
    - 编译笔记本 `compile_notebook`
        - 方法1：`compile_notebook(笔记本::IpynbNotebook)`
            - 功能：将「Jupyter笔记本结构」编译成Julia源码（字符串）
            - 返回：`String`（源码字符串）
        - 方法2：`compile_notebook(输入路径::String, 输出路径::String="$输入路径.jl")`
            - 功能：从**指定路径**读取并编译Jupyter笔记本
            - 返回：写入输出路径的字节数
- 📌提供【解析并直接运行Jupyter笔记本】的方式（视作Julia代码执行）
    - 解析单元格 `parse_cell`
        - 方法 `parse_cell(单元格::IpynbCell)`
            - 功能：将【单个】单元格内容编译解析成Julia表达式（`Expr`对象）
        - 方法 `parse_cell(单元格列表::Vector{IpynbCell})`
            - 功能：将【多个】单元格内容分别编译后【合并】，然后解析成Julia表达式（`Expr`对象）
        - 返回
            - Julia表达式（若为`code`代码类型）
            - `nothing`（若为其它类型）
    - 解析笔记本 `parse_notebook`
        - 等效于「编译笔记本的**所有单元格**」
    - 执行单元格 `eval_cell`
        - 等效于「【解析】并【执行】单元格」
    - 执行笔记本 `eval_notebook`
        - 等效于「【解析】并【执行】笔记本」
        - 逐单元格版本：`eval_notebook_by_cell`
    - 引入笔记本 `include_notebook`
        - 逐单元格版本：`include_notebook_by_cell`

✨创新点：**使用多样的「特殊注释」机制，让使用者能更灵活、更便捷地编译Jupyter笔记本，并能将其【交互式】优势用于库的编写之中**

### 重要机制：单元格「特殊注释」

简介：单元格的主要「特殊注释」及其作用（以`# 单行注释` `#= 块注释 =#`为例）

- `# %ignore-line` 忽略下一行
- `# %ignore-below` 忽略下面所有行
- `# %ignore-cell` 忽略整个单元格
- `# %ignore-begin` 块忽略开始
- `# %ignore-end` 块忽略结束
- `#= %only-compiled` 仅编译后可用（头）
- `%only-compiled =#` 仅编译后可用（尾）
- `# %include <路径>` 引入指定路径的文件内容，替代一整行注释

✨**该笔记本自身**，就是一个好的用法参考来源

#### 各个「特殊注释」的用法

##### 忽略单行

📌简要用途：忽略**下一行**代码

编译前@笔记本单元格：

```julia
[[上边还会被编译]]
# %ignore-line # 忽略单行（可直接在此注释后另加字符，会被一并忽略）
[这是一行被忽略的代码]
[[下边也要被编译]]
```

编译后：

```julia
[[上边还会被编译]]
[[下边也要被编译]]
```

##### 忽略下面所有行

编译前@笔记本单元格

```julia
[[上边的代码正常编译]]
# %ignore-below # 忽略下面所有行（可直接在此注释后另加字符，会被一并忽略）
[
    之后会忽略很多代码
]
[包括这一行]
[不管多长都会被忽略]
```

编译后：

```julia
[[上边的代码正常编译]]
```

##### 忽略整个单元格

编译前@笔记本单元格：

```julia
[上边的代码会被忽略（不会被编译）]
# %ignore-cell # 忽略整个单元格（可直接在此注释后另加字符，会被一并忽略）
[下面的代码也会被忽略]
[⚠️另外，这些代码连着单元格都不会出现在编译后的文件中，连「标识头」都没有]
```

编译后：

```julia
```

↑空字串

📌一般习惯将 `# %ignore-cell` 放在第一行

##### 忽略代码块

📝即「块忽略」

编译前@笔记本单元格：

```julia
[[上边的代码正常编译]]
# %ignore-begin # 开始块忽略（可直接在此注释后另加字符，会被一并忽略）
[这一系列中间的代码会被忽略]
[不管有多少行，除非遇到终止注释]
# %ignore-end # 结束块忽略（可直接在此注释后另加字符，会被一并忽略）
[[下面的代码都会被编译]]
```

编译后：

```julia
[[上边的代码正常编译]]
[[下面的代码都会被编译]]
```

##### 仅编译后可用

主要用途：包装 `module` 等代码，实现编译后模块上下文

- ⚠️对于 **Python** 等【依赖缩进定义上下文】的语言，难以进行此类编译

编译前@笔记本单元格：

```julia
[[上边的代码正常编译，并且会随着笔记本一起执行]]
#= %only-compiled # 开始「仅编译后可用」（可直接在此注释后另加字符，会被一并忽略）
[
    这一系列中间的代码
    - 在「执行笔记本」时被忽略（因为在Julia块注释之中）
    - 但在编译后「上下注释」被移除
        - 因此会在编译后被执行
]
%only-compiled =# # 结束「仅编译后可用」（可直接在此注释后另加字符，会被一并忽略）
[[下面的代码正常编译，并且会随着笔记本一起执行]]
```

编译后：

```julia
[[上边的代码正常编译，并且会随着笔记本一起执行]]
[
    这一系列中间的代码
    - 在「执行笔记本」时被忽略（因为在Julia块注释之中）
    - 但在编译后「上下注释」被移除
        - 因此会在编译后被执行
]
[[下面的代码正常编译，并且会随着笔记本一起执行]]
```

##### 文件引入

主要用途：结合「仅编译后可用」实现「外部代码内联」

- 如：集成某些**中小型映射表**，整合零散源码文件……

编译前@笔记本单元格：

```julia
const square_map_dict = # 这里的等号可以另起一行
# % include to_include.jl 
# ↑ 上面一行会被替换成数据
```

编译前@和笔记本**同目录**下的`to_include.jl`中：
↓文件末尾有换行符

```julia
# 这是一个要被引入的外部字典对象
Dict([
    1 => 1
    2 => 4
    3 => 9
    # ...
])
```

编译后：

```julia
const square_map_dict = # 这里的等号可以另起一行
# 这是一个要被引入的外部字典对象
Dict([
    1 => 1
    2 => 4
    3 => 9
    # ...
])
# ↑ 上面一行会被替换成数据
```

📝Julia的「空白符无关性」允许在等号后边大范围附带注释的空白

## 参考

- 本Julia库的灵感来源：[Promises.jl/src/notebook.jl](https://github.com/fonsp/Promises.jl/blob/main/src/notebook.jl)
    - 源库使用了 [**Pluto.jl**](https://github.com/fonsp/Pluto.jl) 的「笔记本导出」功能
- **Jupyter Notebook** 文件格式（JSON）：[🔗nbformat.readthedocs.io](https://nbformat.readthedocs.io/en/latest/format_description.html#notebook-file-format)

<!-- README-end -->

⚠️该单元格首行注释用于截止生成`README.md`（包括自身）

## 建立模块上下文

📌使用 `# %only-compiled` 控制 `module` 代码，生成模块上下文

In [148]:
# ! ↓这后边注释的代码只有在编译后才会被执行
# ! 使用多行注释/块注释的语法，
# !     以`#= %only-compiled`行*开头*
# !     以`%only-compiled =#`行*结尾*
#= %only-compiled # * ←这个仅需作为前缀（⚠️这注释会被一并移除）
"""
IpynbCompile 主模块
"""
module IpynbCompile # 后续编译后会变为模块上下文
%only-compiled =# # * ←左边同理（⚠️这注释会被一并移除）

## 模块前置

导入库

In [149]:
import JSON

预置语法糖

In [150]:
"JSON常用的字典"
const JSONDict{ValueType} = Dict{String,ValueType} where ValueType

"默认解析出来的JSON字典（与`JSONDict`有本质不同，会影响到后续方法分派，并可能导致歧义）"
const JSONDictAny = JSONDict{Any}

JSONDictAny

## 读取ipynb文件

In [151]:
#= %only-compiled # ! 模块上下文：导出元素
export read_ipynb_json
%only-compiled =#

"""
读取ipynb JSON文件
- @param path .ipynb文件路径
- @return .ipynb文件内容（JSON文本→Julia对象）
"""
read_ipynb_json(path) = open(path, "r") do f
    read(f, String) |> JSON.parse
end

# ! ↓使用`# %ignore-line`让 编译器/解释器 忽略下一行
# %ignore-line
notebook_json = read_ipynb_json("IpynbCompile.ipynb")

Dict{String, Any} with 4 entries:
  "cells"          => Any[Dict{String, Any}("cell_type"=>"markdown", "source"=>…
  "nbformat_minor" => 2
  "metadata"       => Dict{String, Any}("language_info"=>Dict{String, Any}("fil…
  "nbformat"       => 4

## 解析文件元信息

Jupyter Notebook元数据 格式参考

```yaml
{
    "metadata": {
        "kernel_info": {
            # if kernel_info is defined, its name field is required.
            "name": "the name of the kernel"
        },
        "language_info": {
            # if language_info is defined, its name field is required.
            "name": "the programming language of the kernel",
            "version": "the version of the language",
            "codemirror_mode": "The name of the codemirror mode to use [optional]",
        },
    },
    "nbformat": 4,
    "nbformat_minor": 0,
    "cells": [
        # list of cell dictionaries, see below
    ],
}
```

当前Julia笔记本数据：
```json
{
    "language_info": {
        "file_extension": ".jl",
        "mimetype": "application/julia",
        "name": "julia",
        "version": "1.9.1"
    },
    "kernelspec": {
        "name": "julia-1.9",
        "display_name": "Julia 1.9.1",
        "language": "julia"
    }
}
```

In [152]:
# %ignore-cell # ! ←使用`# %ignore-line`让 编译器/解释器 忽略整个单元格
# * ↑建议放在第一行
# ! ⚠️该代码不能有其它冗余的【前缀】字符

let metadata = notebook_json["metadata"],
    var"metadata.language_info" = metadata["language_info"]
    var"metadata.kernelspec" = metadata["kernelspec"]
    @info "notebook_json" notebook_json
    @info "notebook_json.metadata" metadata
    @info "metadata[...]" var"metadata.language_info" var"metadata.kernelspec"
end

┌ Info: notebook_json
│   notebook_json =
│    Dict{String, Any} with 4 entries:
│      "cells"          => Any[Dict{String, Any}("cell_type"=>"markdown", "sour…
│      "nbformat_minor" => 2
│      "metadata"       => Dict{String, Any}("language_info"=>Dict{String, Any}…
└      "nbformat"       => 4
┌ Info: notebook_json.metadata
│   metadata =
│    Dict{String, Any} with 2 entries:
│      "language_info" => Dict{String, Any}("file_extension"=>".jl", "mimetype"…
└      "kernelspec"    => Dict{String, Any}("name"=>"julia-1.9", "display_name"…
┌ Info: metadata[...]
│   metadata.language_info =
│    Dict{String, Any} with 4 entries:
│      "file_extension" => ".jl"
│      "mimetype"       => "application/julia"
│      "name"           => "julia"
│      "version"        => "1.9.1"
│   metadata.kernelspec =
│    Dict{String, Any} with 3 entries:
│      "name"         => "julia-1.9"
│      "display_name" => "Julia 1.9.1"
└      "language"     => "julia"


定义「笔记本」结构

In [153]:
#= %only-compiled # ! 模块上下文：导出元素
export IpynbNotebook, IpynbNotebookMetadata
%only-compiled =#

"""
定义一个Jupyter Notebook的metadata结构
- 🎯规范化存储Jupyter Notebook的元数据
    - 根据官方文档，仅存储【已经确定存在】的「语言信息」和「内核信息」
"""
@kwdef struct IpynbNotebookMetadata # !【2024-01-14 16:09:35】目前只发现这两种信息
    "语言信息"
    language_info::JSONDict{String}
    "内核信息"
    kernelspec::JSONDict{String}
end

"""
定义一个Jupyter Notebook的notebook结构
- 🎯规范化存储Jupyter Notebook的整体数据
"""
@kwdef struct IpynbNotebook{Cell}
    "单元格（类型后续会定义）"
    cells::Vector{Cell}
    "元信息"
    metadata::IpynbNotebookMetadata
    "笔记本格式"
    nbformat::Int
    "笔记本格式（最小版本？）"
    nbformat_minor::Int
end

"""
从JSON到notebook结构
- @method IpynbNotebook{Any}(json) Any泛型：直接保存原始字典
- @method IpynbNotebook{Cell}(json) where {Cell} 其它指定类型：调用`Cell`进行转换
"""
IpynbNotebook{Any}(json) = IpynbNotebook{Any}(;
    cells=json["cells"], # * Any类型→直接保存
    metadata=IpynbNotebookMetadata(json["metadata"]),
    nbformat=json["nbformat"],
    nbformat_minor=json["nbformat_minor"],
)
IpynbNotebook{Cell}(json) where {Cell} = IpynbNotebook{Cell}(;
    cells=Cell.(json["cells"]), # * ←广播类型转换
    metadata=IpynbNotebookMetadata(json["metadata"]),
    nbformat=json["nbformat"],
    nbformat_minor=json["nbformat_minor"],
)

# %ignore-begin # ! ←通过「begin-end」对使用「块忽略」（精确到行）
# ! ↓下面这行仅为测试用，后续将重定向到特制的「笔记本单元格」类型
IpynbNotebook(json) = IpynbNotebook{Any}(json)
# * 这段注释也不会出现在编译后的代码中
# %ignore-end

# 从指定文件加载
IpynbNotebook(ipynb_path::AbstractString) = ipynb_path |> read_ipynb_json |> IpynbNotebook

"""
从JSON到「notebook元数据」结构
"""
IpynbNotebookMetadata(json::JSONDict) = IpynbNotebookMetadata(;
    language_info=json["language_info"],
    kernelspec=json["kernelspec"],
)

# ! ↓使用`# %ignore-below`让 编译器/解释器 忽略后续内容
# %ignore-below
notebook_raw_cell = IpynbNotebook(notebook_json)
notebook_metadata = notebook_raw_cell.metadata
@info "JSON转译结构化成功！" notebook_raw_cell notebook_metadata

┌ Info: JSON转译结构化成功！
│   notebook_raw_cell = IpynbNotebook{Any}(Any[Dict{String, Any}("cell_type" => "markdown", "source" => Any["# IpynbCompile.jl: 一个实用的「Jupyter笔记本→Julia源码」转换小工具"], "metadata" => Dict{String, Any}()), Dict{String, Any}("cell_type" => "markdown", "source" => Any["<!-- README-ignored -->\n", "（✨执行其中所有单元格，可自动构建、测试并生成相应`.jl`源码！）"], "metadata" => Dict{String, Any}()), Dict{String, Any}("cell_type" => "markdown", "source" => Any["## 主要功能"], "metadata" => Dict{String, Any}()), Dict{String, Any}("cell_type" => "markdown", "source" => Any["### 简介"], "metadata" => Dict{String, Any}()), Dict{String, Any}("cell_type" => "markdown", "source" => Any["📍主要功能：**编译转换**&**解释执行** [***Jupyter***](https://jupyter.org/) 笔记本（`.ipynb`文件）\n", "\n", "- 📌可【打开】并【解析】Jupyter笔记本：提供基本的「Jupyter笔记本」「Jupyter笔记本元数据」「Jupyter笔记本单元格」数据结构定义\n", "    - 笔记本 `IpynbNotebook{单元格类型}`\n", "    - 元数据 `IpynbNotebookMetadata`\n", "    - 单元格 `IpynbCell`\n", "- 📌可将Jupyter笔记本（`.ipynb`文件）【转换】成可直接执行的 [***Julia***](https://

notebook编译/头部注释
- 🎯标注 版本信息
- 🎯标注 各类元数据

In [154]:
"""
【内部】从Notebook生成头部注释
- ⚠️末尾有换行
@example IpynbNotebook{Any, IpynbNotebookMetadata}(#= ... =#, IpynbNotebookMetadata(Dict("file_extension" => ".jl", "mimetype" => "application/julia", "name" => "julia", "version" => "1.9.1"), Dict("name" => "julia-1.9", "display_name" => "Julia 1.9.1", "language" => "julia")), 4, 2)
将生成如下代码
```julia
# %% Jupyter Notebook | Julia 1.9.1 @ julia | format 2~4
# % language_info: {"file_extension":".jl","mimetype":"application/julia","name":"julia","version":"1.9.1"}
# % kernelspec: {"name":"julia-1.9","display_name":"Julia 1.9.1","language":"julia"}
# % nbformat: 4
# % nbformat_minor: 2
```
"""
compile_notebook_head(notebook::IpynbNotebook; kwargs...) = """\
# %% Jupyter Notebook | $(notebook.metadata.kernelspec["display_name"]) \
@ $(notebook.metadata.language_info["name"]) | \
format $(notebook.nbformat_minor)~$(notebook.nbformat)
# % language_info: $(JSON.json(notebook.metadata.language_info))
# % kernelspec: $(JSON.json(notebook.metadata.kernelspec))
# % nbformat: $(notebook.nbformat)
# % nbformat_minor: $(notebook.nbformat_minor)
"""

# %ignore-below
# ! ↑使用`# %ignore-below`让 编译器/解释器 忽略后续内容
@assert compile_notebook_head(notebook_raw_cell) == """\
# %% Jupyter Notebook | Julia 1.9.1 @ julia | format 2~4
# % language_info: {"file_extension":".jl","mimetype":"application/julia","name":"julia","version":"1.9.1"}
# % kernelspec: {"name":"julia-1.9","display_name":"Julia 1.9.1","language":"julia"}
# % nbformat: 4
# % nbformat_minor: 2
"""
compile_notebook_head(notebook_raw_cell) |> print

# %% Jupyter Notebook | Julia 1.9.1 @ julia | format 2~4
# % language_info: {"file_extension":".jl","mimetype":"application/julia","name":"julia","version":"1.9.1"}
# % kernelspec: {"name":"julia-1.9","display_name":"Julia 1.9.1","language":"julia"}
# % nbformat: 4
# % nbformat_minor: 2


## 解析处理单元格

定义「单元格」结构

In [155]:
#= %only-compiled # ! 模块上下文：导出元素
export IpynbCell
%only-compiled =#

"""
定义一个Jupyter Notebook的cell结构
- 🎯规范化存储Jupyter Notebook的单元格数据
"""
struct IpynbCell
    cell_type::String
    source::Vector{String}
    metadata::JSONDict
    output::Any

    "基于关键字参数的构造函数"
    IpynbCell(;
       cell_type="code",
       source=String[],
       metadata=JSONDictAny(),
       output=nothing
    ) = new(
        cell_type,
        source,
        metadata,
        output
    )

    """
    自单元格JSON对象的转换
    - 🎯将单元格转换成规范形式：类型+代码+元数据+输出
    - 不负责后续的 编译/解释 预处理
    """
    IpynbCell(json_cell::JSONDict) = IpynbCell(; (
        field => json_cell[string(field)]
        for field::Symbol in fieldnames(IpynbCell)
        if haskey(json_cell, string(field)) # ! 不论JSON对象是否具有：没有⇒报错
    )...)
end

# ! 在此重定向，以便后续外部调用
"重定向「笔记本」的默认「单元格」类型"
IpynbNotebook(json) = IpynbNotebook{IpynbCell}(json)

# %ignore-below
notebook = IpynbNotebook{IpynbCell}(notebook_json)
cells = notebook.cells

76-element Vector{IpynbCell}:
 IpynbCell("markdown", ["# IpynbCompile.jl: 一个实用的「Jupyter笔记本→Julia源码」转换小工具"], Dict{String, Any}(), nothing)
 IpynbCell("markdown", ["<!-- README-ignored -->\n", "（✨执行其中所有单元格，可自动构建、测试并生成相应`.jl`源码！）"], Dict{String, Any}(), nothing)
 IpynbCell("markdown", ["## 主要功能"], Dict{String, Any}(), nothing)
 IpynbCell("markdown", ["### 简介"], Dict{String, Any}(), nothing)
 IpynbCell("markdown", ["📍主要功能：**编译转换**&**解释执行** [***Jupyter***](https://jupyter.org/) 笔记本（`.ipynb`文件）\n", "\n", "- 📌可【打开】并【解析】Jupyter笔记本：提供基本的「Jupyter笔记本」「Jupyter笔记本元数据」「Jupyter笔记本单元格」数据结构定义\n", "    - 笔记本 `IpynbNotebook{单元格类型}`\n", "    - 元数据 `IpynbNotebookMetadata`\n", "    - 单元格 `IpynbCell`\n", "- 📌可将Jupyter笔记本（`.ipynb`文件）【转换】成可直接执行的 [***Julia***](https://julialang.org/) 代码\n", "    - 编译单元格 `compile_cell`\n", "    - 编译笔记本 `compile_notebook`\n", "        - 方法1：`compile_notebook(笔记本::IpynbNotebook)`\n"  …  "            - `nothing`（若为其它类型）\n", "    - 解析笔记本 `parse_notebook`\n", "        - 等效于「编译笔记本的**所

编译/入口

In [156]:
#= %only-compiled # ! 模块上下文：导出元素
export compile_cell
%only-compiled =#

"""
【入口】将一个单元格编译成Julia代码（包括注释）
- 📌根据「单元格类型」`code_type`字段进行细致分派
- ⚠️编译生成的字符串需要附带【完整】的换行信息
    - 亦即：编译后的「每一行」都需附带换行符
"""
compile_cell(cell::IpynbCell; kwargs...)::String = compile_cell(
    # 使用`Val`类型进行分派
    Val(Symbol(cell.cell_type)), 
    # 传递单元格对象自身
    cell;
    # 传递其它附加信息（如单元格序号，后续被称作「行号」）
    kwargs...
)

"""
【入口】将多个单元格编译成Julia代码（包括注释）
- 先各自编译，然后join(_, '\\n')
- ⚠️编译后不附带「最终换行符」
"""
compile_cell(cells::Vector{IpynbCell}; kwargs...)::String = join((
    compile_cell(
        # 传递单元格对象
        cell;
        # 附加单元格序号
        line_num,
        # 传递其它附加信息（如单元格序号，后续被称作「行号」）
        kwargs...
    )
    for (line_num, cell) in enumerate(cells) # ! ←一定是顺序遍历
), '\n')

"""
【内部】对整个单元格的「类型标头」编译
- 🎯生成一行注释，标识单元格
    - 指定单元格的边界
    - 简略说明单元格的信息
- ⚠️生成的代码附带末尾换行符

@example IpynbCell("markdown", ["# IpynbCompile.jl: 一个通用的「Julia Jupyter Notebook→Julia源码文件」小工具"], Dict{String, Any}(), nothing)
（行号为1）将生成
```julia
# %% [1] markdown
```
# ↑末尾附带换行符
"""
compile_cell_head(cell::IpynbCell; kwargs...) = """\
# %% \
$(#= 可选的行号 =# haskey(kwargs, :line_num) ? "[$(kwargs[:line_num])] " : "")\
$(cell.cell_type)
""" # ! ←末尾附带换行符

# %ignore-below
@assert compile_cell_head(notebook.cells[1]) == "# %% markdown\n"
@assert compile_cell_head(notebook.cells[1]; line_num=1) == "# %% [1] markdown\n"

编译/Markdown

In [157]:
"""
对Markdown的编译
- 📌主要方法：转换成多个单行注释

@example IpynbCell("markdown", ["# IpynbCompile.jl: 一个通用的「Julia Jupyter Notebook→Julia源码文件」小工具"], Dict{String, Any}(), nothing)
（行号为1）将被转换为
```julia
# %% [1] markdown
# # IpynbCompile.jl: 一个通用的「Julia Jupyter Notebook→Julia源码文件」小工具
```
# ↑末尾附带换行符
"""
compile_cell(::Val{:markdown}, cell::IpynbCell; kwargs...) = """\
$(#= 附带标头 =# compile_cell_head(cell; kwargs...))\
$(join(
    "# $md_line"
    for md_line in cell.source
) #= ←此处无需附加换行符，`md_line`已自带 =#)
""" # ! ↑末尾附带换行符

# %ignore-line
compile_cell(cells[1]; line_num = 1) |> println

# %% [1] markdown
# # IpynbCompile.jl: 一个实用的「Jupyter笔记本→Julia源码」转换小工具



编译/代码

In [158]:
# %ignore-cell
codes = filter(cells) do cell
    cell.cell_type == "code"
end

21-element Vector{IpynbCell}:
 IpynbCell("code", ["# ! ↓这后边注释的代码只有在编译后才会被执行\n", "# ! 使用多行注释/块注释的语法，\n", "# !     以`#= %only-compiled`行*开头*\n", "# !     以`%only-compiled =#`行*结尾*\n", "#= %only-compiled # * ←这个仅需作为前缀（⚠️这注释会被一并移除）\n", "\"\"\"\n", "IpynbCompile 主模块\n", "\"\"\"\n", "module IpynbCompile # 后续编译后会变为模块上下文\n", "%only-compiled =# # * ←左边同理（⚠️这注释会被一并移除）"], Dict{String, Any}(), nothing)
 IpynbCell("code", ["import JSON"], Dict{String, Any}(), nothing)
 IpynbCell("code", ["\"JSON常用的字典\"\n", "const JSONDict{ValueType} = Dict{String,ValueType} where ValueType\n", "\n", "\"默认解析出来的JSON字典（与`JSONDict`有本质不同，会影响到后续方法分派，并可能导致歧义）\"\n", "const JSONDictAny = JSONDict{Any}"], Dict{String, Any}(), nothing)
 IpynbCell("code", ["#= %only-compiled # ! 模块上下文：导出元素\n", "export read_ipynb_json\n", "%only-compiled =#\n", "\n", "\"\"\"\n", "读取ipynb JSON文件\n", "- @param path .ipynb文件路径\n", "- @return .ipynb文件内容（JSON文本→Julia对象）\n", "\"\"\"\n", "read_ipynb_json(path) = open(path, \"r\") do f\n", "    read(f,

In [159]:
"""
对代码的编译
- @param cell 所需编译的单元格
- @param kwargs 其它附加信息（如行号）
- 📌主要方法：逐行拼接代码，并
    - 📍每行代码的末尾都有换行符，除了最后一行代码

@example IpynbCell("code", ["\"JSON常用的字典\"\n", "const JSONDict{ValueType} = Dict{String,ValueType} where ValueType\n", "\n", "\"默认解析出来的JSON字典（与`JSONDict`有本质不同，会影响到后续方法分派，并可能导致歧义）\"\n", "const JSONDictAny = JSONDict{Any}"], Dict{String, Any}(), nothing)
（行号为1）将被转换为
```julia
"JSON常用的字典"
const JSONDict{ValueType} = Dict{String,ValueType} where ValueType

"默认解析出来的JSON字典（与`JSONDict`有本质不同，会影响到后续方法分派，并可能导致歧义）"
const JSONDictAny = JSONDict{Any}
```
↑⚠️注意：最后一行后自动添加了换行符
"""
function compile_cell(::Val{:code}, cell::IpynbCell; kwargs...)
    code::Union{Nothing,String} = compile_code_lines(cell; kwargs...)
    # 对应「忽略整个单元格」的情形，返回空字串
    isnothing(code) && return ""
    return """\
    $(#= 附带标头 =# compile_cell_head(cell; kwargs...))\
    $code\
    """ # ! ↑编译后的`code`已在最后一行带有换行符
end

"""
【内部，默认不导出】编译代码行
- 🎯根据单元格的`source::Vector{String}`字段，预处理并返回【修改后】的源码
- 📌在此开始执行各种「行编译逻辑」（在此列举部分）- `# %ignore-line` 忽略下一行
    - `# %ignore-below` 忽略下面所有行
    - `# %ignore-cell` 忽略整个单元格
    - `#= %only-compiled` 仅编译后可用（头）
    - `%only-compiled =#` 仅编译后可用（尾）
- ⚠️编译后的文本是「每行都有换行符」
    - 对最后一行增加了换行符，以便和先前所有行一致
- @param cell 所需编译的单元格
- @param kwargs 其它附加信息（如行号）
- @return 编译后的源码 | nothing（表示「完全不呈现单元格」）
"""
function compile_code_lines(cell::IpynbCell;
    # 根路径（默认为「执行编译的文件」所在目录）
    root_path::AbstractString=@__DIR__,
    # 其它参数
    kwargs...)

    local lines::Vector{String} = cell.source
    local len_lines = length(lines)
    local current_line_i::Int = firstindex(lines)
    local current_line::String
    local result::String = ""

    while current_line_i <= len_lines
        current_line = lines[current_line_i]
        # * `%ignore-line` 忽略下一行 | 仅需为行前缀
        if startswith(current_line, "# %ignore-line")
            current_line_i += 1 # ! 结合后续递增，跳过下面一行，不让本「特殊注释」行被编译
        # * `%ignore-below` 忽略下面所有行 | 仅需为行前缀
        elseif startswith(current_line, "# %ignore-below")
            break # ! 结束循环，不再编译后续代码
        # * `%ignore-cell` 忽略整个单元格 | 仅需为行前缀
        elseif startswith(current_line, "# %ignore-cell")
            return nothing # ! 返回「不编译单元格」的信号
        # * `%include` 读取其所指定的路径，并将其内容作为「当前行」添加（不会自动添加换行！） | 仅需为行前缀
        elseif startswith(current_line, "# %include")
            # 在指定的「根路径」参数下行事 # * 无需使用`@inline`，编译器会自动内联
            local relative_path = current_line[nextind(current_line, 1, length("# %include ")):end] |> rstrip # ! ←注意`%include`后边有个空格
            # 读取内容
            local content::String = read(joinpath(root_path, relative_path), String)
            result *= content # ! 不会自动添加换行！
        # * `%ignore-begin` 跳转到`%ignore-end`的下一行，并忽略中间所有行 | 仅需为行前缀
        elseif startswith(current_line, "# %ignore-begin")
            # 只要后续没有以"# %ignore-end"开启的行，就不断跳过
            while !startswith(lines[current_line_i], "# %ignore-end") && current_line_i <= len_lines
                current_line_i += 1 # 忽略性递增
            end # ! 让最终递增跳过"# %ignore-end"所在行
        # * `%only-compiled` 仅编译后可用（多行） | 仅需为行前缀
        elseif (
            startswith(current_line, "#= %only-compiled") ||
            startswith(current_line, "%only-compiled =#")
            )
            # ! 不做任何事情，跳过当前行
        # * 否则：直接将行追加到结果
        else
            result *= current_line
        end
        
        # 最终递增
        current_line_i += 1
    end

    # 最后返回所有行 # ! ↓对最后一行增加换行符，以便和先前所有行一致
    return result * "\n"
end

# %ignore-below

let 引入路径 = "%include.test.jl",
    预期引入内容 = read(引入路径, String),
    引入后内容 = compile_code_lines(
        IpynbCell(; 
            cell_type="code", 
            source=["# %include $引入路径"]
        )
    )
    @assert rstrip(引入后内容) == 预期引入内容
    println(引入后内容)
end

printstyled("↓现在预览下其中所有代码的部分\n"; bold=true, color=:light_green)

# * ↓现在预览下其中所有代码的部分
compile_cell(codes) |> print

# 这是一段会被`# %include`引入编译后笔记本的内容
println("Hello World")

↓现在预览下其中所有代码的部分
# %% [1] code
# ! ↓这后边注释的代码只有在编译后才会被执行
# ! 使用多行注释/块注释的语法，
# !     以`#= %only-compiled`行*开头*
# !     以`%only-compiled =#`行*结尾*
"""
IpynbCompile 主模块
"""
module IpynbCompile # 后续编译后会变为模块上下文


# %% [2] code
import JSON

# %% [3] code
"JSON常用的字典"
const JSONDict{ValueType} = Dict{String,ValueType} where ValueType

"默认解析出来的JSON字典（与`JSONDict`有本质不同，会影响到后续方法分派，并可能导致歧义）"
const JSONDictAny = JSONDict{Any}

# %% [4] code
export read_ipynb_json

"""
读取ipynb JSON文件
- @param path .ipynb文件路径
- @return .ipynb文件内容（JSON文本→Julia对象）
"""
read_ipynb_json(path) = open(path, "r") do f
    read(f, String) |> JSON.parse
end

# ! ↓使用`# %ignore-line`让 编译器/解释器 忽略下一行



# %% [6] code
export IpynbNotebook, IpynbNotebookMetadata

"""
定义一个Jupyter Notebook的metadata结构
- 🎯规范化存储Jupyter Notebook的元数据
    - 根据官方文档，仅存储【已经确定存在】的「语言信息」和「内核信息」
"""
@kwdef struct IpynbNotebookMetadata # !【2024-01-14 16:09:35】目前只发现这两种信息
    "语言信息"
    language_info::JSONDict{Stri

## 解析执行单元格

🎯将单元格解析**编译**成Julia表达式，并可直接作为代码执行
- 【核心】解释：`parse_cell`
    - 📌基本是`compile_cell` ∘ `Meta.parse`的复合
    - 对无法执行的单元格 ⇒ return `nothing`
        - 如markdown单元格
    - 可执行单元格 ⇒ Expr
        - 如code单元格
- 执行：`eval_cell`
    - 📌基本是`parse_cell` ∘ `eval`的复合
    - ⚙️可任意指定其中的`eval`函数

In [160]:
#= %only-compiled # ! 模块上下文：导出元素
export parse_cell, tryparse_cell, eval_cell
%only-compiled =#

"""
解析一个单元格
- 🎯将单元格解析成Julia表达式
- 📌使用`Meta.parseall`解析代码
    - `Meta.parse`只能解析一个Julia表达式
    - 可能会附加上不必要的「:toplevel」表达式
@param cell 单元格
@param parse_function 解析函数（替代原先`Meta.parseall`的位置）
@param kwargs 附加参数
@return 解析后的Julia表达式 | nothing（不可执行）
"""
function parse_cell(cell::IpynbCell; parse_function = Meta.parseall, kwargs...)

    # 只有类型为 code 才执行解析
    cell.cell_type == "code" && return parse_function(
        compile_cell(cell; kwargs...)
    )

    # ! 默认不可执行
    return nothing
end

"""
解析一系列单元格
@param cells 单元格序列
@param parse_function 解析函数（替代原先`Meta.parseall`的位置）
@param kwargs 附加参数
@return 解析后的Julia表达式 | nothing（不可执行）
"""
function parse_cell(cells::Vector{IpynbCell}; parse_function = Meta.parseall, kwargs...)

    # 只有类型为 code 才执行解析
    return parse_function(
        # 预先编译所有代码单元格，然后连接成一个字符串
        join(
            compile_cell(cell; kwargs...)
            for cell in cells
            # 只有类型为`code`的单元格才执行解析
            if cell.cell_type == "code"
        )
    )

    # ! 默认不可执行
    return nothing
end

"""
尝试解析单元格
- 📌用法同`parse_cell`，但会在解析报错时返回`nothing`
    - ⚠️此中「解析报错」≠「解析过程出现错误」
        - 📝解析错误的代码会被`Meta.parseall`包裹进类似`Expr(错误)`的表达式中
        - 例如：`Expr(:incomplete, "incomplete: premature end of input")`
"""
tryparse_cell(args...; kwargs...) = try
    parse_cell(args...; kwargs...)
catch e
    @warn e
    nothing
end

"""
执行单元格
- 🎯执行解析后的单元格（序列）
- @param code_or_codes 单元格 | 单元格序列
- @param eval_function 执行函数（默认为`eval`）
- @param kwargs 附加参数
- @return 执行后表达式的值
"""
eval_cell(code_or_codes; eval_function=eval, kwargs...) = eval_function(
    parse_cell(code_or_codes; kwargs...)
)

# %ignore-below

# 执行其中一个代码单元格 # * 参考「预置语法糖」
eval_cell(codes[3])::Base.Docs.Binding

# 尝试对每个单元格进行解析
[
    tryparse_cell(cell; line_num=i)
    for (i, cell) in enumerate(codes)
]

21-element Vector{Expr}:
 :($(Expr(:toplevel, :(#= none:6 =#), :($(Expr(:incomplete, "incomplete: premature end of input"))))))
 :($(Expr(:toplevel, :(#= none:2 =#), :(import JSON))))
 :($(Expr(:toplevel, :(#= none:2 =#), :(#= none:2 =# Core.@doc "JSON常用的字典" const JSONDict{ValueType} = (Dict{String, ValueType} where ValueType)), :(#= none:5 =#), :(#= none:5 =# Core.@doc "默认解析出来的JSON字典（与`JSONDict`有本质不同，会影响到后续方法分派，并可能导致歧义）" const JSONDictAny = JSONDict{Any}))))
 :($(Expr(:toplevel, :(#= none:2 =#), :(export read_ipynb_json), :(#= none:4 =#), :(#= none:4 =# Core.@doc "读取ipynb JSON文件\n- @param path .ipynb文件路径\n- @return .ipynb文件内容（JSON文本→Julia对象）\n" read_ipynb_json(path) = begin
              #= none:9 =#
              open(path, "r") do f
                  #= none:10 =#
                  read(f, String) |> JSON.parse
              end
          end))))
 :($(Expr(:toplevel)))
 :($(Expr(:toplevel, :(#= none:2 =#), :(export IpynbNotebook, IpynbNotebookMetadata), :(#= none:4 =#), :(#= none:4 

In [161]:
# %ignore-cell
tryparse_cell(codes)

:($(Expr(:toplevel, :(#= none:6 =#), :(#= none:6 =# Core.@doc "IpynbCompile 主模块\n" module IpynbCompile
      #= none:9 =#
      #= none:12 =#
      import JSON
      #= none:14 =#
      #= none:14 =# Core.@doc "JSON常用的字典" const JSONDict{ValueType} = (Dict{String, ValueType} where ValueType)
      #= none:17 =#
      #= none:17 =# Core.@doc "默认解析出来的JSON字典（与`JSONDict`有本质不同，会影响到后续方法分派，并可能导致歧义）" const JSONDictAny = JSONDict{Any}
      #= none:20 =#
      export read_ipynb_json
      #= none:22 =#
      #= none:22 =# Core.@doc "读取ipynb JSON文件\n- @param path .ipynb文件路径\n- @return .ipynb文件内容（JSON文本→Julia对象）\n" read_ipynb_json(path) = begin
                  #= none:27 =#
                  open(path, "r") do f
                      #= none:28 =#
                      read(f, String) |> JSON.parse
                  end
              end
      #= none:34 =#
      export IpynbNotebook, IpynbNotebookMetadata
      #= none:36 =#
      #= none:36 =# Core.@doc "定义一个Jupyter Notebook的metadata结构\n- 🎯规范化

## 编译笔记本

In [162]:
#= %only-compiled # ! 模块上下文：导出元素
export compile_notebook
%only-compiled =#

"""
编译整个笔记本
- 🎯编译整个笔记本对象，形成相应Julia代码
- 📌整体文本：头部注释+各单元格编译（逐个join(_, '\\n')）
- ⚠️末尾不会附加换行符
- @param notebook 要编译的笔记本对象
- @return 编译后的文本
"""
compile_notebook(notebook::IpynbNotebook; kwargs...) = """\
$(compile_notebook_head(notebook; kwargs...))
$(compile_cell(notebook.cells; kwargs...))
""" # ! `$(compile_notebook_head(notebook))`在原本的换行下再空一行，以便与后续单元格分隔

"""
编译整个笔记本，并【写入】指定路径
- @param notebook 要编译的笔记本对象
- @param path 要写入的路径
- @return 写入结果
"""
compile_notebook(notebook::IpynbNotebook, path::AbstractString; kwargs...) = write(
    # 使用 `write`函数，自动写入编译结果
    path, 
    # 传入前编译
    compile_notebook(notebook; kwargs...)
)

"""
编译指定路径的笔记本，并写入指定路径
- 「写入路径」默认为「读入路径+`.jl`」
- @param path 要读取的路径
- @return 写入结果
"""
compile_notebook(path::AbstractString, destination="$path.jl"; kwargs...) = compile_notebook(
    # 直接使用构造函数加载笔记本
    IpynbNotebook(path), 
    # 保存在目标路径
    destination;
    # 其它附加参数 #
    # 自动从`path`构造编译根目录
    root_path=dirname(path),
)

# %ignore-below
compile_notebook(notebook) |> print

# %% Jupyter Notebook | Julia 1.9.1 @ julia | format 2~4
# % language_info: {"file_extension":".jl","mimetype":"application/julia","name":"julia","version":"1.9.1"}
# % kernelspec: {"name":"julia-1.9","display_name":"Julia 1.9.1","language":"julia"}
# % nbformat: 4
# % nbformat_minor: 2

# %% [1] markdown
# # IpynbCompile.jl: 一个实用的「Jupyter笔记本→Julia源码」转换小工具

# %% [2] markdown
# <!-- README-ignored -->
# （✨执行其中所有单元格，可自动构建、测试并生成相应`.jl`源码！）

# %% [3] markdown
# ## 主要功能

# %% [4] markdown
# ### 简介

# %% [5] markdown
# 📍主要功能：**编译转换**&**解释执行** [***Jupyter***](https://jupyter.org/) 笔记本（`.ipynb`文件）
# 
# - 📌可【打开】并【解析】Jupyter笔记本：提供基本的「Jupyter笔记本」「Jupyter笔记本元数据」「Jupyter笔记本单元格」数据结构定义
#     - 笔记本 `IpynbNotebook{单元格类型}`
#     - 元数据 `IpynbNotebookMetadata`
#     - 单元格 `IpynbCell`
# - 📌可将Jupyter笔记本（`.ipynb`文件）【转换】成可直接执行的 [***Julia***](https://julialang.org/) 代码
#     - 编译单元格 `compile_cell`
#     - 编译笔记本 `compile_notebook`
#         - 方法1：`compile_notebook(笔记本::IpynbNotebook)`
#             - 功能：将「Jupyt

## 解析执行笔记本

执行笔记本

In [163]:
#= %only-compiled # ! 模块上下文：导出元素
export eval_notebook, eval_notebook_by_cell
%only-compiled =#

"""
【整个】解释并执行Jupyter笔记本
- 📌直接使用`eval_cell`对笔记本的所有单元格进行解释执行
    - 可以实现一些「编译后可用」的「上下文相关代码」
        - 如「将全笔记本代码打包成一个模块」
"""
eval_notebook(notebook::IpynbNotebook; kwargs...) = eval_cell(
   notebook.cells;
   # 其它附加参数（如「编译根目录」）
   kwargs...
)

"""
【逐单元格】解释并执行Jupyter笔记本
- 📌逐个取出并执行笔记本中的代码
- 记录并返回最后一个返回值
"""
function eval_notebook_by_cell(notebook::IpynbNotebook; kwargs...)
    # 返回值
    local result::Any = nothing
    # 逐个执行
    for cell in notebook.cells
        result = eval_cell(cell; kwargs...)
    end
    # 返回最后一个结果
    return result
end

# ! 测试代码放在最后边

eval_notebook_by_cell

引入笔记本

In [164]:
#= %only-compiled # ! 模块上下文：导出元素
export include_notebook, include_notebook_by_cell
%only-compiled =#

"""
从【字符串】路径解析并【整个】编译执行整个笔记本的代码
- 📌「执行笔记本」已经有`eval_notebook`支持了
- 🎯直接解析并执行`.ipynb`文件
- 📌先加载并编译Jupyter笔记本，再【整个】执行其所有单元格
- 会像`include`一样返回「最后一个执行的单元格的返回值」
"""
include_notebook(path::AbstractString; kwargs...) = eval_notebook(
    path |> 
    read_ipynb_json |> 
    IpynbNotebook{IpynbCell};
    # 其它附加参数（如「编译根目录」）
    kwargs...
)

"""
解析并【逐单元格】执行整个笔记本的代码
- 📌「执行笔记本」已经有`eval_notebook_by_cell`支持了
- 🎯直接解析并执行`.ipynb`文件
- 📌先加载并编译Jupyter笔记本，再【逐个】执行其单元格
- ⚠️不会记录单元格执行的【中间】返回值
    - 但正如`include`一样，「最后一个执行的单元格的返回值」仍然会被返回
"""
include_notebook_by_cell(path::AbstractString; kwargs...) = eval_notebook_by_cell(
    path |> 
    read_ipynb_json |> 
    IpynbNotebook{IpynbCell};
    # 其它附加参数（如「编译根目录」）
    kwargs...
)

# %ignore-below

# * 递回执行自身代码（自举）
SELF_FILE = "IpynbCompile.ipynb"
include_notebook(SELF_FILE)

# 检验是否成功导入
@assert @isdefined IpynbCompile # ! 模块上下文生效：所有代码现在都在模块之中
printstyled("✅Jupyter笔记本文件引入完成，模块导入成功！\n"; color=:light_green, bold=true)
@show IpynbCompile
println()

# * 打印导出的所有符号
printstyled("📜以下为IpynbCompile模块导出的所有$(length(names(IpynbCompile)))个符号：\n"; color=:light_blue, bold=true)
for name in names(IpynbCompile)
    println(name)
end

✅Jupyter笔记本文件引入完成，模块导入成功！
IpynbCompile = Main.IpynbCompile

📜以下为IpynbCompile模块导出的所有14个符号：
IpynbCell
IpynbCompile
IpynbNotebook
IpynbNotebookMetadata
compile_cell
compile_notebook
eval_cell
eval_notebook
eval_notebook_by_cell
include_notebook
include_notebook_by_cell
parse_cell
read_ipynb_json
tryparse_cell


## 关闭模块上下文

In [165]:
# ! ↓这后边注释的代码只有在编译后才会被执行
# ! 仍然使用多行注释语法，以便统一格式
#= %only-compiled
end # module
%only-compiled =#

## 自动构建

构建过程主要包括：

- **自举**构建主模块，生成库文件
- 扫描`src`目录下基本所有Jupyter笔记本（`.ipynb`），编译生成`.jl`源码
- 提取该文件开头Markdown笔记，在**项目根目录**下**生成自述文件**（`README.md`）
    - 因此`README.md`暂且只有一种语言（常更新的语言）

⚠️不应该在编译后的库文件中看到任何代码

In [166]:
# %ignore-cell # * 自举构建主模块
# * 自编译生成`.jl`源码
OUT_LIB_FILE = "IpynbCompile.jl" # 直接作为库的主文件
write_bytes = compile_notebook(SELF_FILE, OUT_LIB_FILE)
printstyled("✅Jupyter笔记本「主模块」自编译成功！\n（共写入 $write_bytes 个字节）\n"; color=:light_yellow, bold=true)

✅Jupyter笔记本「主模块」自编译成功！
（共写入 30326 个字节）


In [167]:
# %ignore-cell # * 扫描`src`目录，自动构建主模块
# * 📝Julia 文件夹遍历：`walkdir`迭代器
# * 🔗参考：参考：https://stackoverflow.com/questions/58258101/how-to-loop-through-a-folder-of-sub-folders-in-julia
function compile_all_notebook_in_dir(root_folder)

    local path::AbstractString, new_path::AbstractString

    # 遍历所有文件
    for (root, dirs, file_names) in walkdir(root_folder)
        for file_name in file_names
            path = joinpath.(root, file_name)
            if isdir(path)
                # * 为路径⇒递归深入
                compile_all_notebook_in_dir(path)
            elseif endswith(path, ".ipynb")
                # * 为Jupyter笔记本（`*.ipynb`）⇒编译
                if file_name !== SELF_FILE # * 无需再编译自身
                    new_path = replace(path, ".ipynb" => ".jl")
                    compile_notebook(
                        path,
                        # 替换末尾扩展名
                        new_path;
                        # ! 根目录后续会由`path`自行指定
                    )
                    printstyled("Compiled: $path => $new_path\n", color=:light_green, bold=true)
                end
            end
        end
    end
end

PATH_SRC = "."
compile_all_notebook_in_dir(PATH_SRC)

Compiled: .\compiler.ipynb => .\compiler.jl


In [168]:
# %ignore-cell # * 扫猫自身Markdown单元格，自动生成`README.md`
"决定「单元格采集结束」的标识"
FLAG_END = "<!-- README-end -->"
FLAG_IGNORE = "<!-- README-ignored -->"

# * 过滤Markdown单元格
markdowns = filter(notebook.cells) do cell
    cell.cell_type == "markdown"
end
# * 截取Markdown单元格 | 直到开头有`FLAG_END`标记的行（不考虑换行符）
README_END_INDEX = findlast(markdowns) do cell
    !isempty(cell.source) && startswith(cell.source[begin], FLAG_END)
end
README_markdowns = markdowns[begin:README_END_INDEX-1]

# * 提取Markdown代码，聚合生成原始文档
README_markdown_TEXT = join((
    join(cell.source) * '\n' # ←这里需要加上换行
    for cell in README_markdowns
    # 根据【空单元格】或【首行注释】进行忽略
    if !(isempty(cell.source) || startswith(cell.source[begin], FLAG_IGNORE))
), '\n')

# * 继续处理：缩进4→2，附加注释
README_markdown_TEXT = join((
    begin
        local space_stripped_line = lstrip(line, ' ')
        local head_space_length = length(line) - length(space_stripped_line)
        # 缩进缩减到原先的一半
        ' '^(head_space_length ÷ 2) * space_stripped_line
    end
    for line in split(README_markdown_TEXT, '\n')
), '\n')
using Dates: now # * 增加日期注释（不会在正文显示）
README_markdown_TEXT = """\
<!-- ⚠️该文件由 `$SELF_FILE` 自动生成于 $(now())，无需手动修改 -->
$README_markdown_TEXT\
"""
print(README_markdown_TEXT)

"上一级路径（执行的时候是在`src`目录下）"
ROOT_PATH = "./../"
README_FILE = "README.md"
write(joinpath(ROOT_PATH, README_FILE), README_markdown_TEXT)

<!-- ⚠️该文件由 `IpynbCompile.ipynb` 自动生成于 2024-01-16T16:00:12.739，无需手动修改 -->
# IpynbCompile.jl: 一个实用的「Jupyter笔记本→Julia源码」转换小工具

## 主要功能

### 简介

📍主要功能：**编译转换**&**解释执行** [***Jupyter***](https://jupyter.org/) 笔记本（`.ipynb`文件）

- 📌可【打开】并【解析】Jupyter笔记本：提供基本的「Jupyter笔记本」「Jupyter笔记本元数据」「Jupyter笔记本单元格」数据结构定义
  - 笔记本 `IpynbNotebook{单元格类型}`
  - 元数据 `IpynbNotebookMetadata`
  - 单元格 `IpynbCell`
- 📌可将Jupyter笔记本（`.ipynb`文件）【转换】成可直接执行的 [***Julia***](https://julialang.org/) 代码
  - 编译单元格 `compile_cell`
  - 编译笔记本 `compile_notebook`
    - 方法1：`compile_notebook(笔记本::IpynbNotebook)`
      - 功能：将「Jupyter笔记本结构」编译成Julia源码（字符串）
      - 返回：`String`（源码字符串）
    - 方法2：`compile_notebook(输入路径::String, 输出路径::String="$输入路径.jl")`
      - 功能：从**指定路径**读取并编译Jupyter笔记本
      - 返回：写入输出路径的字节数
- 📌提供【解析并直接运行Jupyter笔记本】的方式（视作Julia代码执行）
  - 解析单元格 `parse_cell`
    - 方法 `parse_cell(单元格::IpynbCell)`
      - 功能：将【单个】单元格内容编译解析成Julia表达式（`Expr`对象）
    - 方法 `parse_cell(单元格列表::Vector{IpynbCell})`
      - 功能：将【多个】单元格内容分别编译后【合并】，然后解析成Julia表达式（

7213